In [27]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last_expr'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import numpy as np
import pandas as pd
import os
import csv
import shutil

# Create annotation task - IDFG

Find images that are labeled as containing animals and copy them to one folder and zip it.

In [3]:
eval_dir = '/home/yasiyu/yasiyu_temp/IDFG/IDFG_20190104/eval'

In [4]:
folders = ['Beaverhead_elk', 'St.Joe_elk', 'ClearCreek_mustelids']
num_to_sample = {
    'Beaverhead_elk': None, 
    'St.Joe_elk': 5000,
    'ClearCreek_mustelids': 5000,
}

Include all 31,621 images from Beaverhead_elk, and sample 5k from St_Joe_elk and ClearCreek_mustelids each.

In [5]:
detection_threshold = 0.9

In [6]:
animal_images = []
all_false_negatives = []
all_false_positives = []
for folder in folders:
    folder_clean = folder.replace('.', '_')
    csv_name = 'eval_IDFG_{}.csv'.format(folder_clean)
    df = pd.read_csv(os.path.join(eval_dir, csv_name))
    print(csv_name)
    print(df.shape)
    samples = df[df.label == True]['image_id']
    print('number of animals:', len(samples))
    
    samples = samples
    if num_to_sample[folder]:
        n_samples = num_to_sample[folder]
        samples = samples.sample(n_samples)
    print('number of samples to include:', len(samples))
    animal_images.extend(sorted(samples))
    
#    gt_positives = set(rows)
    
#     rows = df[df.pred > detection_threshold]['image_id']
#     print('number of preds above 0.9 conf:', len(rows))
#     preds_positives = set(rows)

#     false_negatives = gt_positives.difference(preds_positives) # in gt_positives but not in preds_positives
#     false_positives = preds_positives.difference(gt_positives) # in preds_positives but not in gt_positives
#     print('number of false negatives:', len(false_negatives))
#     print('number of false positives:', len(false_positives))
#     all_false_negatives.append(false_negatives)
#     all_false_positives.append(false_positives)
    
    print()

eval_IDFG_Beaverhead_elk.csv
(239910, 3)
number of animals: 31621
number of samples to include: 31621

eval_IDFG_St_Joe_elk.csv
(239006, 3)
number of animals: 103394
number of samples to include: 5000

eval_IDFG_ClearCreek_mustelids.csv
(199954, 3)
number of animals: 140968
number of samples to include: 5000



In [7]:
len(animal_images)

41621

Add images from the other two folders that have high detection confidence.

In [8]:
results_dir = '/home/yasiyu/yasiyu_temp/IDFG/IDFG_20190104/results2'
other_folders = ['Statewide_wolf', 'Focal_lion', 'CrowCreek_moose']

In [9]:
num_images_to_take = 5000

for folder in other_folders:
    folder_clean = folder.replace('.', '_')
    csv_name = 'IDFG_{}.csv'.format(folder_clean)
    df = pd.read_csv(os.path.join(results_dir, csv_name))
    
    detection_threshold = 0.9
    if folder == 'CrowCreek_moose':
        detection_threshold = 0.99
    
    samples = df[df.max_confidence > detection_threshold]['image_path']
    print('folder:', folder)
    print('detection_threshold:', detection_threshold)
    print('number of high confidence detection images:', len(samples))
    
    if len(samples) > num_images_to_take:
        samples = samples.sample(num_images_to_take)
    
    print('number of samples included: {}'.format(len(samples)))
    animal_images.extend(sorted(samples))
    print()

folder: Statewide_wolf
detection_threshold: 0.9
number of high confidence detection images: 552627
number of samples included: 5000

folder: Focal_lion
detection_threshold: 0.9
number of high confidence detection images: 97965
number of samples included: 5000

folder: CrowCreek_moose
detection_threshold: 0.99
number of high confidence detection images: 18056
number of samples included: 5000



In [13]:
len(animal_images)

56621

In [12]:
animal_images[-6000]

'Focal_lion/56/101RECNX/IMG_1912.JPG'

In [ ]:
save_path = '/home/yasiyu/yasiyu_temp/IDFG/IDFG_20190104/IDFG_20190104_images_to_annotate.csv'
with open(save_path, 'w') as f:
    writer = csv.writer(f)
    for image_id in animal_images:
        writer.writerow([image_id])

## Copy images to local and then zip them

In [17]:
with open(save_path, 'r') as f:
    reader = csv.reader(f)
    animal_images = list(reader)

In [18]:
len(animal_images)

56621

In [19]:
animal_images[-6000]

['Focal_lion/56/101RECNX/IMG_1912.JPG']

In [21]:
dest = '/datadrive/IDFG/IDFG_20190104_images_to_annotate'
origin = '/home/yasiyu/mnt/wildlifeblobssc/idfg'

In [ ]:
failed_images = []

for i, image_id in enumerate(animal_images):
    image_id = image_id[0]
    dest_image_id = image_id.replace('/', '*')
    
    origin_path = os.path.join(origin, image_id)
    dest_path = os.path.join(dest, dest_image_id)
    
    if not os.path.exists(dest_path):
        try:
            shutil.copyfile(origin_path, dest_path)
        except Exception as e:
            print('Image {}, exception: {}'.format(image_id, str(e)))
            failed_images.append(image_id)
    
    if i % 2000 == 0:
        print(i/56621)

In [ ]:
!azcopy \
    --source /home/yasiyu/yasiyu_temp/IDFG/IDFG_20190104/IDFG_20190104_images_to_annotate.csv \
    --destination https://ai4edevshare.file.core.windows.net/ai4edevfs/annotations/outgoing_requests/IDFG_20190104_images_to_annotate.csv \
    --dest-key 